Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

Functions

In [ ]:
def read_file(file_path):
    dataset = pd.read_csv(file_path)
    
    X_train = dataset.iloc[1:2100, 3:-1].values
    y_train = dataset.iloc[1:2100, 1].values
    
    img_names = dataset.iloc[2100:3500,2].values
    
    X_test = dataset.iloc[2100:3500, 3:-1].values
    y_test = dataset.iloc[2100:3500, 1].values
    
    x_cross_validation = dataset.iloc[2600:3300, 3:-1].values
    y_cross_validation = dataset.iloc[2600:3300, 1].values
    
    return X_train,y_train,X_test , y_test, x_cross_validation,y_cross_validation,img_names

In [ ]:
def get_best_parameter(X_train,y_train,x_cross_validation,y_cross_validation):
    # cross validation set(to determine the parameters)
    Accuracy = 0
    estimator = 0
    for i in range(1,300):
        regressor = RandomForestClassifier(n_estimators=i, random_state=0)
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(x_cross_validation)
        new_accuracy = round(accuracy_score(y_cross_validation, y_pred) * 100,2)
        if Accuracy < new_accuracy :
            Accuracy = new_accuracy
            estimator = i
    return estimator

In [ ]:
def Print_to_file(file_name,y_pred):
    with open(file_name, "a") as o:
            o.write(str(y_pred) + "\n")
    

In [ ]:
def get_Classification(X_train,y_train,x_test,y_test,estimator,img_names):
    pridiction_Image = []
    test_image=[]
    male_indecate=0
    female_indecate=0
    line_classifier=[]
    image_classifier=[]
    starttime=0
    count = 0
    stop_watch = 0
    prev_img = ""
    regressor = RandomForestClassifier(n_estimators=estimator, random_state=0)
    regressor.fit(X_train, y_train)
    print(len(x_test))
    print(len(img_names))
    
    for img in img_names :
        if img != prev_img :
            prev_img = img
            male_indecate=0
            female_indecate=0
            if count != 0 :
                test_image.append(y_test[count - 1])
                for line in image_classifier :
                    if int(line) == 1 :
                        male_indecate+=1
                    else :
                        female_indecate +=1
                if male_indecate > female_indecate :
                    Print_to_file("result.txt",1)
                    pridiction_Image.append(1)
                else :
                    Print_to_file("result.txt",0)
                    pridiction_Image.append(0)
                stop_watch += round(time.time()-starttime,2)
                Print_to_file("time.txt",round(time.time()-starttime,2))


            image_classifier=[]
            starttime=time.time()
        y_pred = regressor.predict(x_test[count].reshape(1,-1))
        image_classifier.append(y_pred)
        line_classifier.append(y_pred)
        count += 1
    
    print("Accuracy = " + str(round(accuracy_score(y_test, line_classifier) * 100,2))+ "% for line by line")
    print("Accuracy = " + str(round(accuracy_score(test_image, pridiction_Image) * 100,2))+ "% for img by img")
    print("time = " , stop_watch)
    return line_classifier

In [ ]:
x_train,y_train,X_test , y_test, x_cross_validation,y_cross_validation,img_names = read_file("Dataset.csv")
estimator = get_best_parameter(x_train,y_train,x_cross_validation,y_cross_validation)
get_Classification(x_train,y_train,X_test,y_test,estimator,img_names)